## Generating new text based on training data with LSTM-networks, LYRICS GENERATION VIA COLAB

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Note: Since this approach will use a bidirectional LSTM-network, the training is twice as slow and requires twice as much memory

Note: Since textual context depends on both the previous and future words, bidirectional layers are needed to capture the context better.

**Tip: before you have pre-processed your lyrics dataset, it's probably better to run your notebook in CPU-mode in order to save your free GPU-time in Google Colab!**

---
---
# PART 1 - PROCESS THE DATA

In [4]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow.keras.utils as ku
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

import re


In [ ]:
# I also worked with a lyrics dataset obtained through web scraping, but I couldn't achieve good results with it. I discussed the issues with the teacher after the last lecture and showed him the specific problems I was facing—such as errors caused by missing characters like quotation ("") marks in some lines. For further experiments and better results, I used a different dataset, which produced improved outcomes, as shown in the file."

In [ ]:

# experiments with 2 datasets, with different epochs, Cleaning data with different approaches just commented that one,  run different experiment and mostly found good results with this dataset and i am showing it below in this file, All screenshot that I caputred also putted into screenshot folder as well. 


# Reducing to 50 epochs speeds up training and may reduce overfitting, but risks underfitting and producing lower-quality, less coherent text. It often stops learning before reaching optimal performance.
# Screenshot also available in screenshot folder, there is first 3 to 4 lines works fine but after their its going to pring single or 3 to words only which is really meaningless with 50 numbers of epochs.

# After 150 epochs, the model achieved strong performance with 87.5% accuracy and 0.82 loss, generating coherent and grammatically structured text. The final output shows learned patterns and flow, though minor grammatical flaws remain.


# My first LSTM_poetry_lyrics_generation 01 file, I did same experments with this but overall i found better result, expermients like changing dropout, LSTM(150) etc

#model = Sequential()
#model.add(Embedding(total_words, 100, input_length=max_sequence_len - 1))
#model.add(LSTM(150, return_sequences=True))
#model.add(Dropout(0.2))
#model.add(LSTM(100))  # Stacked LSTM for added learning depth
#model.add(Dropout(0.2))

# The model received "glaxy" as input, which isn't in the vocabulary, resulting in an empty token sequence. As a result, the generation failed or produced invalid output, highlighting the importance of input validation

# Also some time we got one word only and then meaning less statement without repeating words, which seems model is working according to our expecations 


In [ ]:
# Similarities 
# All three assignments employ standard text preprocessing steps such as tokenization, lowercasing, and removal of stop words, LabelEncoder etc
# Further extraction like converting numaric data, check numaric value etc also using same in this project
# Each assignment evaluates model performance using metrics such as accuracy, precision, recall, loss and accuracy.
# More of other side 
# its using more complex model unlike previous models 
# Adds dropout and data augmentation to reduce overfitting and Includes pipeline tools like hyperparameter tuning and checkpointing.

In [6]:
# path to the data file
# path = "/content/drive/MyDrive/Colab Notebooks/ADV_ML/Exercise_3/linkin_park_scraped_lyrics.csv"
path = '/content/drive/MyDrive/Colab Notebooks/PoetryFoundationData.csv'

In [7]:
# load the data and show first rows
df = pd.read_csv(path)
df.head()

,Unnamed: 0,Title,Poem,Poet,Tags
0,0,\r\r\n Objects Used to Prop...,"\r\r\nDog bone, stapler,\r\r\ncribbage board, ...",Michelle Menting,NaN
1,1,\r\r\n The New Church\r\r\n...,"\r\r\nThe old cupola glinted above the clouds,...",Lucia Cherciu,NaN
2,2,\r\r\n Look for Me\r\r\n ...,\r\r\nLook for me under the hood\r\r\nof that ...,Ted Kooser,NaN
3,3,\r\r\n Wild Life\r\r\n ...,"\r\r\nBehind the silo, the Mother Rabbit\r\r\n...",Grace Cavalieri,NaN
4,4,\r\r\n Umbrella\r\r\n ...,\r\r\nWhen I push your button\r\r\nyou fly off...,Connie Wanek,NaN


In [8]:
df.iloc[0]

,0
Unnamed: 0,0
Title,\r\r\n Objects Used to Prop...
Poem,"\r\r\nDog bone, stapler,\r\r\ncribbage board, ..."
Poet,Michelle Menting
Tags,NaN


In [ ]:

# trying to clean the data and remove the mess like \r\n\r\n or \n\r\n\r\n

# import re

# text = df['Poem'][0]

# # Step 1: Replace \r\n\r\n with a real paragraph break
# text = text.replace('\\r\\n\\r\\n', '\n')
# text = text.replace('\\r\\n\\r\\', '\n')

# # Step 2: Replace any remaining \r\n with space (joining broken lines)
# text = text.replace('\\r\\n', ' ')

# # Step 3: Remove extra spaces (optional)
# text = re.sub(r' +', ' ', text)

# # Step 4: Strip leading/trailing whitespace
# text = text.strip()

# print(text)


In [2]:
df.iloc[0][0]
# each sentence of the song is connected with \n\n
# each paragragraph is connected with \n\n\n\n
# these '\n' needs to be cconverted to comma
# lyrics for each singer is mentioned with such text: [Mark Wakefield (Mike Shinoda):] or [Mike Shinoda:]
# some repeating part also mentioned with [8x]
# everthing starting with '[' and ending with ']'needs to be cleaned

NameError: name 'df' is not defined

## **Filter the unneeded parts of the text**

In [ ]:

# Cleaning data but this code does not works properly and seve my purposes.

# # Function to clean lyrics
# def clean_lyrics(text):

# 	# Step 1: Replace \r\n\r\n with a real paragraph break
# 	text = text.replace('\\r\\n\\r\\n', '\n')
# 	text = text.replace('\\r\\n\\r\\', '\n')

# 	# Step 2: Replace any remaining \r\n with space (joining broken lines)
# 	text = text.replace('\\r\\n', ' ')

# 	# Step 3: Remove extra spaces (optional)
# 	text = re.sub(r' +', ' ', text)

# 	# Step 4: Strip leading/trailing whitespace
# 	text = text.strip()

# 	return text

# # Apply the function to the DataFrame
# df['Cleaned_Poem'] = df['Poem'].apply(clean_lyrics)

# # Display the DataFrame
# # print(df[['Lyrics', 'Cleaned_Lyrics']])

# # Display the DataFrame
# df

In [ ]:

# Cleaning data but this code does not works properly and seve my purposes. still we get \r\n\r\n kind of characters in the statement somewhere in the middle though this removed from start or doubles etc

# # Function to clean lyrics
# def clean_lyrics(text):

# 	# Step 1: Decode the escaped characters
# 	text = text.encode('utf-8').decode('unicode_escape')

# 	# Step 2: Remove all double newlines (which came from \r\n\r\n)
# 	text = text.replace('\r\n\r\n', '')

# 	# Optional: Also remove any single \r\n left (joining broken lines)
# 	text = text.replace('\r\n', ' ')

# 	# Step 3: Strip extra spaces
# 	text = re.sub(r' +', ' ', text).strip()

# 	return text

# # Apply the function to the DataFrame
# df['Cleaned_Poem'] = df['Poem'].apply(clean_lyrics)

# # Display the DataFrame
# # print(df[['Lyrics', 'Cleaned_Lyrics']])

# # Display the DataFrame
# df

In [ ]:
# Finally this peace of code works for me at the moment
def clean_lyrics(text):

	# Step 1: Replace double line breaks (\r\n\r\n) with a single space
	text = text.replace('\r\n\r\n', ' ')

	# Step 2: Replace single line breaks (\r\n) with a space
	text = text.replace('\r\n', ' ')

	# Step 3: Normalize multiple spaces into one space
	text = re.sub(r' +', ' ', text)

	# Step 4: Strip leading/trailing spaces
	text = text.strip()

	return text

# Apply the function to the DataFrame
df['Cleaned_Poem'] = df['Poem'].apply(clean_lyrics)

# Display the DataFrame
# print(df[['Lyrics', 'Cleaned_Lyrics']])

# Display the DataFrame
df

,Unnamed: 0,Title,Poem,Poet,Tags,Cleaned_Poem
0,0,\r\r\n Objects Used to Prop...,"\r\r\nDog bone, stapler,\r\r\ncribbage board, ...",Michelle Menting,NaN,"Dog bone, stapler,\r cribbage board, garlic pr..."
1,1,\r\r\n The New Church\r\r\n...,"\r\r\nThe old cupola glinted above the clouds,...",Lucia Cherciu,NaN,"The old cupola glinted above the clouds, shone..."
2,2,\r\r\n Look for Me\r\r\n ...,\r\r\nLook for me under the hood\r\r\nof that ...,Ted Kooser,NaN,Look for me under the hood\r of that old Chevr...
3,3,\r\r\n Wild Life\r\r\n ...,"\r\r\nBehind the silo, the Mother Rabbit\r\r\n...",Grace Cavalieri,NaN,"Behind the silo, the Mother Rabbit\r hunches l..."
4,4,\r\r\n Umbrella\r\r\n ...,\r\r\nWhen I push your button\r\r\nyou fly off...,Connie Wanek,NaN,When I push your button\r you fly off the hand...
...,...,...,...,...,...,...
13849,13,\r\r\n 1-800-FEAR\r\r\n ...,\r\r\nWe'd like to talk with you about ...,Jody Gladding,"Living,Social Commentaries,Popular Culture",We'd like to talk with you about fear they sai...
13850,14,\r\r\n The Death of Atahual...,\r\r\n\r\r\n,William Jay Smith,NaN,
13851,15,\r\r\n Poet's Wish\r\r\n ...,\r\r\n\r\r\n,William Jay Smith,NaN,
13852,0,\r\r\n 0\r\r\n,\r\r\n Philosophic\r\r\nin its comple...,Hailey Leithauser,"Arts & Sciences,Philosophy","Philosophic\r in its complex, ovoid emptiness,..."


In [14]:
df['Poem'][0]

"\r\r\nDog bone, stapler,\r\r\ncribbage board, garlic press\r\r\n     because this window is loose—lacks\r\r\nsuction, lacks grip.\r\r\nBungee cord, bootstrap,\r\r\ndog leash, leather belt\r\r\n     because this window had sash cords.\r\r\nThey frayed. They broke.\r\r\nFeather duster, thatch of straw, empty\r\r\nbottle of Elmer's glue\r\r\n     because this window is loud—its hinges clack\r\r\nopen, clack shut.\r\r\nStuffed bear, baby blanket,\r\r\nsingle crib newel\r\r\n     because this window is split. It's dividing\r\r\nin two.\r\r\nVelvet moss, sagebrush,\r\r\nwillow branch, robin's wing\r\r\n     because this window, it's pane-less. It's only\r\r\na frame of air.\r\r\n"

In [15]:
df['Cleaned_Poem'][0]

"Dog bone, stapler,\r cribbage board, garlic press\r because this window is loose—lacks\r suction, lacks grip.\r Bungee cord, bootstrap,\r dog leash, leather belt\r because this window had sash cords.\r They frayed. They broke.\r Feather duster, thatch of straw, empty\r bottle of Elmer's glue\r because this window is loud—its hinges clack\r open, clack shut.\r Stuffed bear, baby blanket,\r single crib newel\r because this window is split. It's dividing\r in two.\r Velvet moss, sagebrush,\r willow branch, robin's wing\r because this window, it's pane-less. It's only\r a frame of air."

In [16]:
data = df['Cleaned_Poem'].to_list()
data

["Dog bone, stapler,\r cribbage board, garlic press\r because this window is loose—lacks\r suction, lacks grip.\r Bungee cord, bootstrap,\r dog leash, leather belt\r because this window had sash cords.\r They frayed. They broke.\r Feather duster, thatch of straw, empty\r bottle of Elmer's glue\r because this window is loud—its hinges clack\r open, clack shut.\r Stuffed bear, baby blanket,\r single crib newel\r because this window is split. It's dividing\r in two.\r Velvet moss, sagebrush,\r willow branch, robin's wing\r because this window, it's pane-less. It's only\r a frame of air.",
 'The old cupola glinted above the clouds, shone\r among fir trees, but it took him an hour\r for the half mile all the way up the hill. As he trailed,\r the village passed him by, greeted him,\r asked about his health, but everybody hurried\r to catch the mass, left him leaning against fences,\r measuring the road with the walking stick he sculpted.\r He yearned for the day when the new church\r would b

In [17]:
# from wordcloud import WordCloud

# # EDA: Generating WordCloud to visualize
# # the text
# wordcloud = WordCloud(max_font_size=50,
#                       max_words=100,
#                       background_color="black").generate(data)

# # Plotting the WordCloud
# plt.figure(figsize=(8, 4))
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis("off")
# plt.savefig("WordCloud.png")
# plt.show()

In [18]:
# Generating the corpus by
# splitting the text into lines

# Apply .lower() to each item in the list
corpus = [item.lower() for item in data]
print(f'Amount of sentences: {len(corpus)}')
print(corpus[:10])

# # we got 13854 sentences, and Total Words: 132884, so we need to limit to 5000 words by experimenting
corpus = corpus[:100]
print(f'Amount of sentences after drop: {len(corpus)}')

Amount of sentences: 13854
["dog bone, stapler,\r cribbage board, garlic press\r because this window is loose—lacks\r suction, lacks grip.\r bungee cord, bootstrap,\r dog leash, leather belt\r because this window had sash cords.\r they frayed. they broke.\r feather duster, thatch of straw, empty\r bottle of elmer's glue\r because this window is loud—its hinges clack\r open, clack shut.\r stuffed bear, baby blanket,\r single crib newel\r because this window is split. it's dividing\r in two.\r velvet moss, sagebrush,\r willow branch, robin's wing\r because this window, it's pane-less. it's only\r a frame of air.", 'the old cupola glinted above the clouds, shone\r among fir trees, but it took him an hour\r for the half mile all the way up the hill. as he trailed,\r the village passed him by, greeted him,\r asked about his health, but everybody hurried\r to catch the mass, left him leaning against fences,\r measuring the road with the walking stick he sculpted.\r he yearned for the day whe

In [19]:
# Fitting the Tokenizer on the Corpus
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

# Vocabulary count of the corpus
total_words = len(tokenizer.word_index)

# the more unique words we have in the corpus
# the heavier our text generation model is going to be to train and use
print("Total Words:", total_words)


Total Words: 5430


In [20]:
# Converting the text into embeddings (vectorized representations)
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]

	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

# calculate the longest sequence in the data
max_sequence_len = max([len(x) for x in input_sequences])

# based on the longest sequence (text) in the data
# add extra zeroes to shorter texts so that
# all sequences have equal length (this is called padding)
input_sequences = np.array(pad_sequences(input_sequences,
										maxlen=max_sequence_len,
										padding='pre'))

predictors, label = input_sequences[:, :-1], input_sequences[:, -1]
label = ku.to_categorical(label, num_classes=total_words+1)

In [21]:
# Building a Bi-Directional LSTM Model
model = Sequential()

# embedding layer, input = total words, output 100 nodes
# this suggests a huge simplifcation of the corpus
# might be a good idea to increase this, however, it also adds
# complexity
model.add(Embedding(total_words + 1, 100,
					input_length=max_sequence_len-1))

# bidirectional LSTM -> to handle context for both ways
# => context of previous words and context of future words
model.add(Bidirectional(LSTM(150, return_sequences=True)))

# some extra layers for optimization
model.add(Dropout(0.2))
model.add(LSTM(100))

# a dense network with reqularization
# SMALL FIX: wrap the amount of nodes into int(), so
# that the division by 2 does not result in decimals
# amount of nodes has to be an even integer
model.add(Dense(int(total_words+1/2), activation='relu',
				kernel_regularizer=regularizers.l2(0.01)))

# output layer -> every possible word has a separate node for
# probability of that word being the next word in the sentence
model.add(Dense(total_words + 1, activation='softmax'))

# compile the model
model.compile(loss='categorical_crossentropy',
			optimizer='adam', metrics=['accuracy'])

print(model.summary())


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
# history = model.fit(predictors, label, epochs=150, verbose=1)

Epoch 1/150
652/652 ━━━━━━━━━━━━━━━━━━━━ 83s 114ms/step - accuracy: 0.0666 - loss: 7.2680
Epoch 2/150
652/652 ━━━━━━━━━━━━━━━━━━━━ 82s 119ms/step - accuracy: 0.0732 - loss: 6.4962
Epoch 3/150
652/652 ━━━━━━━━━━━━━━━━━━━━ 82s 118ms/step - accuracy: 0.0756 - loss: 6.3293
Epoch 4/150
652/652 ━━━━━━━━━━━━━━━━━━━━ 77s 118ms/step - accuracy: 0.0961 - loss: 6.1807
Epoch 5/150
652/652 ━━━━━━━━━━━━━━━━━━━━ 82s 118ms/step - accuracy: 0.1007 - loss: 6.0901
Epoch 6/150
652/652 ━━━━━━━━━━━━━━━━━━━━ 82s 118ms/step - accuracy: 0.1050 - loss: 5.9743
Epoch 7/150
652/652 ━━━━━━━━━━━━━━━━━━━━ 82s 119ms/step - accuracy: 0.1107 - loss: 5.9110
Epoch 8/150
652/652 ━━━━━━━━━━━━━━━━━━━━ 82s 118ms/step - accuracy: 0.1155 - loss: 5.7787
Epoch 9/150
652/652 ━━━━━━━━━━━━━━━━━━━━ 77s 118ms/step - accuracy: 0.1333 - loss: 5.6418
Epoch 10/150
652/652 ━━━━━━━━━━━━━━━━━━━━ 77s 119ms/step - accuracy: 0.1407 - loss: 5.5279
Epoch 11/150
652/652 ━━━━━━━━━━━━━━━━━━━━ 82s 119ms/step - accuracy: 0.1479 - loss: 5.4417
Epoch 12

In [ ]:
# the starting point of the the model on top of which
# new words are going to be added (similar to prompt in LLMs)
seed_text = "because"

# how many words we wish to generate
next_words = 15
ouptut_text = ""

# loop 25 times to create new words to extend
# the original seed text
for _ in range(next_words):
    # convert the current seed text into vectors + padding
    token_list = tokenizer.texts_to_sequences([seed_text])[0]

    token_list = pad_sequences(
        [token_list], maxlen=max_sequence_len - 1,
        padding='pre')

    # predict the next most probable word based on current seed text
    predicted = np.argmax(model.predict(token_list,
                                        verbose=0), axis=-1)
    # initalize next word
    output_word = ""

    # find the predicted word in the tokenizer
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break

    # finally, add the new predicted word into the end of
    # the current seed text
    seed_text += " " + output_word
    print(seed_text)

# print the result generated by the model
print("\n-- THE FINAL OUTPUT --")
print(seed_text)



# import numpy as np
# from keras.preprocessing.sequence import pad_sequences

# # the starting point of the the model on top of which
# # new words are going to be added (similar to prompt in LLMs)
# original_seed_text = "because"  # <-- replace with any test input
# seed_words = original_seed_text.split()

# # check for unknown words
# unknown_words = [word for word in seed_words if word not in tokenizer.word_index]

# if unknown_words:
#     print(f"[INFO] The following words are not in the vocabulary and will be ignored: {unknown_words}")
#     seed_text = ' '.join([word for word in seed_words if word in tokenizer.word_index])
# else:
#     seed_text = original_seed_text

# if not seed_text:
#     print("[ERROR] All words in the seed text were unknown.")
# else:
#     # how many words we wish to generate
#     next_words = 15
#     output_text = seed_text

#     # loop 15 times to create new words to extend the original seed text
#     for _ in range(next_words):
#         # convert the current seed text into vectors + padding
#         token_list = tokenizer.texts_to_sequences([output_text])[0]
#         token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')

#         # predict the next most probable word
#         predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)

#         # find the predicted word in the tokenizer
#         output_word = ""
#         for word, index in tokenizer.word_index.items():
#             if index == predicted:
#                 output_word = word
#                 break

#         # append the new word to the seed text
#         output_text += " " + output_word

#     # print the final result
#     print("\n-- THE FINAL OUTPUT --")
#     print(output_text)

#     if unknown_words:
#         print(f" Model handled unknown input. Skipped these words: {unknown_words}")
#     else:
#         print(" All seed words were recognized by the tokenizer.")



because you
because you can
because you can the
because you can the artist
because you can the artist were
because you can the artist were he
because you can the artist were he wanted
because you can the artist were he wanted those
because you can the artist were he wanted those hundred
because you can the artist were he wanted those hundred foot
because you can the artist were he wanted those hundred foot niche
because you can the artist were he wanted those hundred foot niche now
because you can the artist were he wanted those hundred foot niche now empty
because you can the artist were he wanted those hundred foot niche now empty but
because you can the artist were he wanted those hundred foot niche now empty but each

-- THE FINAL OUTPUT --
because you can the artist were he wanted those hundred foot niche now empty but each


In [25]:
# probably a good idea to save the model after training and download it somewhere
model.save("Waseem_Poetry_Model.keras")


import pickle

# to save the fitted tokenizer
with open('poetry_gen_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)